<a href="https://colab.research.google.com/github/rubel-47/Projects/blob/main/Bangla_Next_Word_Using_LSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.layers import Embedding, LSTM, Dense
from tensorflow.keras.models import Sequential
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.optimizers import Adam
import pickle
import numpy as np
import os

In [5]:
from google.colab import files
uploaded = files.upload()

Saving Bangla.txt to Bangla.txt


In [6]:
 file = open("Bangla.txt", "r", encoding = "utf8")

# store file in list
lines = []
for i in file:
    lines.append(i)

# Convert list to string
data = ""
for i in lines:
  data = ' '. join(lines) 

#replace unnecessary stuff with space
data = data.replace('\n', '').replace('\r', '').replace('\ufeff', '').replace('“','').replace('”','')  #new line, carriage return, unicode character --> replace by space

#remove unnecessary spaces 
data = data.split()
data = ' '.join(data)
data[:500]

'প্রকল্পসংশ্লিষ্ট সূত্রে জানা গেছে, শুরুতে কমিউনিটি মেডিকেল কলেজ অ্যান্ড হাসপাতালের ১২ কাঠা জমিতে ছয়তলা ভবন করার সিদ্ধান্ত হয়। কিন্তু একাধিকবার প্রকল্প পরিচালক পাল্টায়। মাঝখানে অনেক দিন কোনো প্রকল্প পরিচালক ছিলেন না। কয়েক দফা বৈঠকের পর মূল্যবান জমি বিবেচনায় ভবনের উচ্চতা বাড়িয়ে ৯তলা করার সিদ্ধান্ত হয়। এর ফলে নকশা প্রণয়ন ও খরচ পুনর্নির্ধারণসহ নানা পরিবর্তন আনা হয়। খরচ আগের চেয়ে বেড়ে ২২ কোটি ১২ লাখ ২৬ হাজার টাকায় দাঁড়ায়। একাধিকবার প্রকল্পটি নিয়ে দুই পক্ষের মতের অমিল হয়। শেষ পর্যন্ত নির্মাণকাজ নিয়ে দ'

In [7]:
len(data)

4503

In [8]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts([data])

# saving the tokenizer for predict function
pickle.dump(tokenizer, open('token.pkl', 'wb'))

sequence_data = tokenizer.texts_to_sequences([data])[0]
sequence_data[:15]

[162, 163, 164, 165, 166, 53, 54, 13, 55, 14, 56, 167, 168, 169, 27]

In [6]:
len(sequence_data)

689

In [9]:
vocab_size = len(tokenizer.word_index) + 1
print(vocab_size)

422


In [10]:
sequences = []

for i in range(3, len(sequence_data)):
    words = sequence_data[i-3:i+1]
    sequences.append(words)
    
print("The Length of sequences are: ", len(sequences))
sequences = np.array(sequences)
sequences[:10]

The Length of sequences are:  686


array([[162, 163, 164, 165],
       [163, 164, 165, 166],
       [164, 165, 166,  53],
       [165, 166,  53,  54],
       [166,  53,  54,  13],
       [ 53,  54,  13,  55],
       [ 54,  13,  55,  14],
       [ 13,  55,  14,  56],
       [ 55,  14,  56, 167],
       [ 14,  56, 167, 168]])

In [11]:
X = []
y = []

for i in sequences:
    X.append(i[0:3])
    y.append(i[3])
    
X = np.array(X)
y = np.array(y)

In [12]:
y = to_categorical(y, num_classes=vocab_size)
y[:5]

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)

In [13]:
model = Sequential()
model.add(Embedding(vocab_size, 10, input_length=3))
model.add(LSTM(1000, return_sequences=True))
model.add(LSTM(1000))
model.add(Dense(1000, activation="relu"))
model.add(Dense(vocab_size, activation="softmax"))

In [14]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 3, 10)             4220      
                                                                 
 lstm (LSTM)                 (None, 3, 1000)           4044000   
                                                                 
 lstm_1 (LSTM)               (None, 1000)              8004000   
                                                                 
 dense (Dense)               (None, 1000)              1001000   
                                                                 
 dense_1 (Dense)             (None, 422)               422422    
                                                                 
Total params: 13,475,642
Trainable params: 13,475,642
Non-trainable params: 0
_________________________________________________________________


In [15]:
from tensorflow.keras.callbacks import ModelCheckpoint

checkpoint = ModelCheckpoint("next_words.h5", monitor='loss', verbose=1, save_best_only=True)
model.compile(loss="categorical_crossentropy", optimizer=Adam(learning_rate=0.001))
model.fit(X, y, epochs=70, batch_size=64, callbacks=[checkpoint])

Epoch 1/70
11/11 [==============================] - ETA: 0s - loss: 6.0468
Epoch 1: loss improved from inf to 6.04682, saving model to next_words.h5
11/11 [==============================] - 8s 46ms/step - loss: 6.0468
Epoch 2/70
 9/11 [=======================>......] - ETA: 0s - loss: 6.0203
Epoch 2: loss improved from 6.04682 to 6.02151, saving model to next_words.h5
11/11 [==============================] - 1s 53ms/step - loss: 6.0215
Epoch 3/70
 9/11 [=======================>......] - ETA: 0s - loss: 5.9323
Epoch 3: loss improved from 6.02151 to 5.94573, saving model to next_words.h5
11/11 [==============================] - 0s 48ms/step - loss: 5.9457
Epoch 4/70
 9/11 [=======================>......] - ETA: 0s - loss: 5.9160
Epoch 4: loss improved from 5.94573 to 5.89260, saving model to next_words.h5
11/11 [==============================] - 1s 59ms/step - loss: 5.8926
Epoch 5/70
10/11 [==========================>...] - ETA: 0s - loss: 5.7763
Epoch 5: loss improved from 5.89260 to 5.

In [16]:
from tensorflow.keras.models import load_model
import numpy as np
import pickle

# Load the model and tokenizer
model = load_model('next_words.h5')
tokenizer = pickle.load(open('token.pkl', 'rb'))

def Predict_Next_Words(model, tokenizer, text):

  sequence = tokenizer.texts_to_sequences([text])
  sequence = np.array(sequence)
  preds = np.argmax(model.predict(sequence))
  predicted_word = ""
  
  for key, value in tokenizer.word_index.items():
      if value == preds:
          predicted_word = key
          break
  
  print(predicted_word)
  return predicted_word

In [19]:
while(True):
  text = input("Enter your line: ")
  
  if text == "0":
      print("Execution completed.....")
      break
  
  else:
      try:
          text = text.split(" ")
          text = text[-3:]
          print(text)
        
          Predict_Next_Words(model, tokenizer, text)
          
      except Exception as e:
        print("Error occurred: ",e)
        continue

Enter your line: কোনো কাজ না 
['কাজ', 'না', '']
কারণ
Enter your line:  আবদুর রহমান নামে
['আবদুর', 'রহমান', 'নামে']
একজন
Enter your line: সে ক্ষেত্রে তাদের 
['ক্ষেত্রে', 'তাদের', '']
হতে
Enter your line: নানা পরিবর্তন আনা
['নানা', 'পরিবর্তন', 'আনা']
হয়।
Enter your line: 0
Execution completed.....
